# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
J. Li  ->  J. Li  |  ['J. Li']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
F. Walter  ->  F. Walter  |  ['F. Walter']


Arxiv has 65 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.10632
extracting tarball to tmp_2505.10632...

 done.
  0: tmp_2505.10632/main.tex, 360 lines
  1: tmp_2505.10632/natbib.tex, 96 lines
  2: tmp_2505.10632/natnotes.tex, 332 lines
  3: tmp_2505.10632/aassymbols.tex, 579 lines


Retrieving document from  https://arxiv.org/e-print/2505.10805
extracting tarball to tmp_2505.10805...

/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2505.10632/aassymbols.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2505.11224...

 done.


Found 154 bibliographic references in tmp_2505.11224/references.bbl.
Error retrieving bib data for Fu2024HydrogenExoplanet: 'author'
Error retrieving bib data for Xue20242: 'author'
Retrieving document from  https://arxiv.org/e-print/2505.11263


extracting tarball to tmp_2505.11263... done.


Found 245 bibliographic references in tmp_2505.11263/main.bbl.
Issues with the citations
syntax error in line 1201: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.10908-b31b1b.svg)](https://arxiv.org/abs/2505.10908) | **The impact of spiral arms on the star formation life cycle**  |
|| A. Romanelli, et al. -- incl., <mark>A. Hughes</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-05-19*|
|*Comments*| *12 pages, 5 figures*|
|**Abstract**|            The matter cycle between gas clouds and stars in galaxies plays a crucial role in regulating galaxy evolution through feedback mechanisms. In turn, the local and global galactic environments shape the interstellar medium and provide the initial conditions for star formation, potentially affecting the properties of this small-scale matter cycle. In particular, spiral arms have been proposed to play a pivotal role in the star formation life cycle, by enhancing the gas density and triggering star formation. However, their exact role is still debated. In this paper, we investigate the role of spiral arms in the giant molecular cloud evolutionary life cycle and on the star formation process in a sample of 22 nearby spiral galaxies from the PHANGS survey. We measure the cloud lifetime, the feedback timescale, the typical distance between independent regions and the star formation efficiency in spiral arms and inter-arm regions separately. We find that the distributions of the cloud lifetime as well as the feedback timescale are similar in both environments. This result suggests that spiral arms are unlikely to play a dominant role in triggering star formation. By contrast, the star formation efficiency appears to be slightly higher in inter-arm regions compared to spiral arms.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.11224-b31b1b.svg)](https://arxiv.org/abs/2505.11224) | **Tracing the formation and migration history: molecular signatures in the atmosphere of misaligned hot Jupiter WASP-94Ab using JWST NIRSpec/G395H**  |
|| <mark>E.-M. Ahrer</mark>, et al. |
|*Appeared on*| *2025-05-19*|
|*Comments*| *19 pages, 15 figures, accepted for publication in MNRAS*|
|**Abstract**|            The discovery of hot Jupiters that orbit very close to their host stars has long challenged traditional models of planetary formation and migration. Characterising their atmospheric composition - mainly in the form of the carbon-to-oxygen (C/O) ratio and metallicity - can provide insights into their formation locations and evolution pathways. With JWST we can characterise the atmospheres of these types of planets more precisely than previously possible, primarily because it allows us to determine both their atmospheric oxygen and carbon composition. Here, we present a JWST NIRSpec/G395H transmission spectrum from 2.8-5.1$\mu m$ of WASP-94Ab, an inflated hot Jupiter with a retrograde misaligned orbit around its F-type host star. We find a relatively cloud-free atmosphere, with absorption features of H$_2$O and CO$_2$ at detection significances of $\sim 4\sigma$ and $\sim 11\sigma$, respectively. In addition, we detect tentative evidence of CO absorption at $\sim3\sigma$, as well as hints of sulphur with the detection of H$_2$S at a $\sim 2.5\sigma$ confidence level. Our favoured equilibrium chemistry model determines a C/O ratio of $0.49^{+0.08}_{-0.13}$ for WASP-94Ab's atmosphere, which is substellar compared to the star's C/O ratio of $0.68 \pm 0.10$. The retrieved atmospheric metallicity is similar to the star's metallicity as both are $\sim 2\times$ solar. We find that this sub-stellar C/O ratio and stellar metallicity can be best explained by pebble accretion or planetesimal accretion in combination with large-distance migration of the planet.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.11263-b31b1b.svg)](https://arxiv.org/abs/2505.11263) | **A Cosmic Miracle: A Remarkably Luminous Galaxy at $z_{\rm{spec}}=14.44$ Confirmed with JWST**  |
|| R. P. Naidu, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-05-19*|
|*Comments*| *Submitted to the Open Journal of Astrophysics. Comments greatly appreciated and warmly welcomed!*|
|**Abstract**|            JWST has revealed a stunning population of bright galaxies at surprisingly early epochs, $z>10$, where few such sources were expected. Here we present the most distant example of this class yet -- MoM-z14, a luminous ($M_{\rm{UV}}=-20.2$) source in the COSMOS legacy field at $z_{\rm{spec}}=14.44^{+0.02}_{-0.02}$ that expands the observational frontier to a mere 280 million years after the Big Bang. The redshift is confirmed with NIRSpec/prism spectroscopy through a sharp Lyman-$\alpha$ break and $\approx3\sigma$ detections of five rest-UV emission lines. The number density of bright $z_{\rm{spec}}\approx14-15$ sources implied by our "Mirage or Miracle" survey spanning $\approx350$ arcmin$^{2}$ is $>100\times$ larger ($182^{+329}_{-105}\times$) than pre-JWST consensus models. The high EWs of UV lines (${\approx}15{-}35$ Å) signal a rising star-formation history, with a ${\approx}10\times$ increase in the last 5 Myr ($\rm{SFR_{\rm{5Myr}}}/\rm{SFR_{\rm{50Myr}}}=9.9^{+3.0}_{-5.8}$). The source is extremely compact (circularized $r_{\rm{e}} = 74^{+15}_{-12}$ pc), and yet resolved, suggesting an AGN is not the dominant source of light. The steep UV slope ($\beta=-2.5^{+0.2}_{-0.2}$) implies negligible dust attenuation and a young stellar population. The absence of a strong damping wing may indicate that the immediate surroundings of MoM-z14 are partially ionized at a redshift where virtually every reionization model predicts a $\approx100\%$ neutral fraction. The nitrogen emission and highly super-solar [N/C]$>1$ hint at an abundance pattern similar to local globular clusters that may have once hosted luminous supermassive stars. Since this abundance pattern is also common among the most ancient stars born in the Milky Way, we may be directly witnessing the formation of such stars in dense clusters, connecting galaxy evolution across the entire sweep of cosmic time.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.10632-b31b1b.svg)](https://arxiv.org/abs/2505.10632) | **Measuring Emission Lines with JWST-MegaScience Medium-Bands: A New Window into Dust and Star Formation at Cosmic Noon**  |
|| B. Lorenz, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-05-19*|
|*Comments*| *13 pages, 4 figures*|
|**Abstract**|            We demonstrate the power of JWST-NIRCam medium-band photometry to measure emission line fluxes and study dust and star formation properties of galaxies at cosmic noon. In this work, we present photometric emission line measurements and spatially-resolved maps of H$\alpha$ and Pa$\beta$ for a sample of 14 galaxies at $1.3\leq z\leq 2.4$, observed by the MegaScience medium-band survey and the UNCOVER deep spectroscopic survey. We measure line fluxes directly from the medium-band photometry and compare with spectroscopic measurements from UNCOVER. We find reasonable agreement between the photometric and spectroscopic emission line fluxes for both H$\alpha$ and Pa$\beta$, with scatter $<0.15$ dex down to emission line equivalent widths of $10$Å. We also make a nebular dust measurement from the ratio Pa$\beta$ / H$\alpha$, finding an average nebular A$_\mathrm{V}$ of 1.4. Our photometric A$_\mathrm{V}$ measurements show a slightly larger scatter of $0.5$ magnitudes when compared to spectroscopic measurements; however, this scatter may be partially caused by aperture effects. Finally, we produce spatially resolved maps of H$\alpha$ emission, Pa$\beta$ emission, and stellar continuum. We find that offsets in H$\alpha$ and Pa$\beta$ emission are common, especially for galaxies with the highest A$_\mathrm{V}$, indicating dusty sub-structures. Furthermore, the correlation between H$\alpha$ and continuum emission decreases with increasing A$_\mathrm{V}$, suggesting that the dustiest objects have clumpy dust and star formation distributions. Our study demonstrates the power of medium-band photometry to directly probe emission line strengths, star formation, and dust attenuation for hundreds of galaxies in UNCOVER and thousands of galaxies in upcoming JWST medium-band surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.10805-b31b1b.svg)](https://arxiv.org/abs/2505.10805) | **Frequency and Abundance of Binary sUpermassive bLack holes from Optical Variability Surveys (FABULOVS): Hubble Space Telescope Imaging of Radial-Velocity Selected Binary Candidates**  |
|| L. Nolan, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-05-19*|
|*Comments*| *22 pages, 8 figures, 6 tables, submitted to ApJ*|
|**Abstract**|            Sub-parsec (sub-pc) binary supermassive black holes (BSBHs) should be common from galaxy mergers, yet direct evidence has been elusive. We present HST/WFC3IR F160W imaging for a sample of 8 candidate sub-pc BSBHs at redshifts z~0.1--0.5, as well as cross-comparison with a sample of ordinary quasars with archival HST/WFC3 IR F160W images. These 8 candidate sub-pc BSBHs were identified from multi-epoch spectroscopic surveys of quasars (including both typical quasars and those with single-peaked velocity-offset broad lines). whose broad H$\beta$ lines are significantly offset (by ~< a few hundred km/s) from the systemic redshifts. We directly test the prediction that the host galaxies of BSBHs would have a higher fraction of disturbed morphologies and younger stellar bulges from recent interactions than those of control quasars. After careful subtraction of the central quasar light, our candidate BSBH hosts show a statistically undifferentiated distribution of host asymmetry, indicative of a similar fraction of recent mergers. While a significantly larger sample is needed to place this result on a much firmer statistical ground, it opens questions as to the timescale differences between galaxy merger and BSBH formation, or the efficacy of the radial-velocity-shift--based selection of sub-pc BSBH candidates.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.10915-b31b1b.svg)](https://arxiv.org/abs/2505.10915) | **Precise calculation of the EFT likelihood with primordial non-Gaussianities**  |
|| J.-Y. Ke, <mark>Y. Wang</mark>, P. He |
|*Appeared on*| *2025-05-19*|
|*Comments*| *26 pages, 1 figure*|
|**Abstract**|            We perform a precise calculation of the EFT conditional likelihood for large-scale structure (LSS) via the saddle-point expansion method in the presence of primordial non-Gaussianities (PNG). The precision is manifested at two levels: one corresponding to the consideration of higher-order noise terms, and the other to the inclusion of contributions around the saddle points. In computing the latter, we encounter the same issue of the negative modes as in the context of false vacuum decay, which necessitates deforming the original integration contour into the combination of the steepest descent contours to ensure a convergent and real result. We demonstrate through detailed calculations that, upon the incorporation of leading-order PNG, both types of extensions introduce irreducible field-dependent contributions to the conditional likelihood. This insight motivates the systematic inclusion of additional effective terms within the forward modeling framework. Our work facilitates Bayesian forward modeling under non-Gaussian initial conditions, thereby enabling more stringent constraints on the parameters describing PNG.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.10995-b31b1b.svg)](https://arxiv.org/abs/2505.10995) | **Recurrent Jetlets Associated with the Disappearance of a Satellite Spot**  |
|| L. Yang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-05-19*|
|*Comments*| *22 pages, 9 figures, accepted for publication in The Astrophysical Journal*|
|**Abstract**|            Recurrent small-scale eruptions are fascinating phenomena in the solar atmosphere. However, their underlying physical mechanisms remain unclear. On 2021 May 23, five recurrent jetlets (J1-J5) were observed continuously ejecting from a satellite spot located at the north edge of AR 12824. Using high-resolution, multi-wavelength data from NVST, SDO, and IRIS, we investigate the physical characteristics of these jetlets and their relationship with the satellite spot. The widths of these jetlets range from 1300 to 2900 km, their lifetimes range span 3 to 10 minutes, and their projection speeds vary from 152.8 to 406.0 km s$^{-1}$. During the eruptions, the satellite spot moved northwest at a low speed of 376 $\pm$ 12 m s$^{-1}$. Its area gradually decreased due to magnetic cancellation with surrounding positive magnetic field, resulting in an average cancellation rate of 1.3$\times$10$^{18}$ Mx hr$^{-1}$. Dark lanes that separated from the satellite spot and small pores were observed to move toward nearby these features or dark lanes with opposite polarities, eventually disappearing during the magnetic cancellation process. J4 was driven by an eruption of a micro-filament. Spectral observations revealed a redshift on the right side of J4 and a blueshift on the left side of its base, suggesting a counterclockwise rotation. The horizontal magnetic field of the satellite spot consistently exhibited a vortex structure throughout its evolution until it vanished. The nonlinear force-free field extrapolation confirms that the satellite spot serves as one footpoint of a mini-flux rope. These observations reveal that these jetlets might result from three-dimensional null-point magnetic reconnection, initiated by the continuous eruption of a mini-flux-rope or multiple mini-flux-ropes, driven by sustained magnetic cancellation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.10908/./images/sfr_m_dens.png', 'tmp_2505.10908/./images/four_panels.png', 'tmp_2505.10908/./images/scatter_plot.png']
copying  tmp_2505.10908/./images/sfr_m_dens.png to _build/html/
copying  tmp_2505.10908/./images/four_panels.png to _build/html/
copying  tmp_2505.10908/./images/scatter_plot.png to _build/html/
exported in  _build/html/2505.10908.md
    + _build/html/tmp_2505.10908/./images/sfr_m_dens.png
    + _build/html/tmp_2505.10908/./images/four_panels.png
    + _build/html/tmp_2505.10908/./images/scatter_plot.png
found figures ['tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_Tiberius_Eureka_exotic_R100_with_residuals.png', 'tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_Tiberius_Eureka_exotic_R400_with_residuals.png', 'tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_models_with_contributions.png', 'tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_models_with_contributions_eq_chem.png', 'tmp_2505.11224/./figures/WASP-

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Halpha}{H\alpha}$
$\newcommand{\HII}{H\textsc{ii}}$
$\newcommand{\Htwo}{H_2}$
$\newcommand{\mic}{\mum}$
$\newcommand{\arraystretch}{1.25}$</div>



<div id="title">

# The impact of spiral arms on the star formation life cycle

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.10908-b31b1b.svg)](https://arxiv.org/abs/2505.10908)<mark>Appeared on: 2025-05-19</mark> -  _12 pages, 5 figures_

</div>
<div id="authors">

A. Romanelli, et al. -- incl., <mark>A. Hughes</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** The matter cycle between gas clouds and stars in galaxies plays a crucial role in regulating galaxy evolution through feedback mechanisms. In turn, the local and global galactic environments shape the interstellar medium and provide the initial conditions for star formation, potentially affecting the properties of this small-scale matter cycle. In particular, spiral arms have been proposed to play a pivotal role in the star formation life cycle, by enhancing the gas density and triggering star formation. However, their exact role is still debated. In this study, we investigated the role of spiral arms in the giant molecular cloud evolutionary life cycle and on the star formation process in a sample of 22 nearby spiral galaxies from the PHANGS survey. We measured the cloud lifetime, the feedback timescale, the typical distance between independent regions, and the star formation efficiency in spiral arms and inter-arm regions separately. We find that the distributions of the cloud lifetime as well as the feedback timescale are similar in both environments. This result suggests that spiral arms are unlikely to play a dominant role in triggering star formation. By contrast, the star formation efficiency appears to be slightly higher in inter-arm regions compared to spiral arms.

</div>

<div id="div_fig1">

<img src="tmp_2505.10908/./images/sfr_m_dens.png" alt="Fig2" width="100%"/>

**Figure 2. -** Molecular gas surface density versus SFR surface density averaged over the area enclosed by arm and inter-arm regions. Dashed lines mark the position of the median of each sample. (*ksrelation*)

</div>
<div id="div_fig2">

<img src="tmp_2505.10908/./images/four_panels.png" alt="Fig4" width="100%"/>

**Figure 4. -** Cumulative distribution function of the cloud lifetime (top left), feedback timescale (top right), region separation length (bottom left), and SFE (bottom right). Each panel shows the c.d.f. for the parameters calculated in spiral arms (orange) and inter-arm regions (blue), as well as for the full galaxies (red,  ([Kim, Chevance and Kruijssen 2022]()) ) as a thick solid line, and the smoothed c.d.f as a shaded line. In cases where only an upper limit could be determined, the value indicated in Table \ref{table1} is used. (*cdf*)

</div>
<div id="div_fig3">

<img src="tmp_2505.10908/./images/scatter_plot.png" alt="Fig5" width="100%"/>

**Figure 5. -** Cloud lifetime (top left), feedback timescale (top right), region separation length (bottom left), and integrated SFE (bottom right) measured through our statistical analysis. For each parameter, we report the values calculated in the spiral arms on the x-axis and in the inter-arm regions on the y-axis for each galaxy. The grey line shows the one-to-one relation. Each data point is colour-coded according to the averaged value obtained by [Kim, Chevance and Kruijssen (2022)]() for the full galaxy. For the feedback timescale and the region separation length, upper limits in the spiral arms are indicated with arrows. (*scatter*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.10908"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\revision}[1]{#1}$
$\newcommand{\Rsun}{R_{\odot}}$
$\newcommand{\Msun}{M_{\odot}}$
$\newcommand{\Rjup}{R_{\mathrm{J}}}$
$\newcommand{\Mjup}{M_{\mathrm{J}}}$
$\newcommand{\Rp}{R_{\mathrm{P}}}$
$\newcommand{\Mp}{M_{\mathrm{P}}}$
$\newcommand{\Rs}{R_{\mathrm{*}}}$
$\newcommand{\Ms}{R_{\mathrm{*}}}$
$\newcommand{\Teq}{T_{\mathrm{eq}}}$
$\newcommand{\Teff}{T_{\mathrm{eff}}}$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\obliquity}{|\lambda|}$
$\newcommand{\mystar}{WASP-94A\xspace}$
$\newcommand{\myplanet}{WASP-94A b\xspace}$
$\newcommand{\eureka}{\texttt{Eureka!}\xspace}$
$\newcommand{\tiberius}{\texttt{Tiberius}\xspace}$
$\newcommand{\exoticjedi}{\texttt{ExoTiC-JEDI}\xspace}$
$\newcommand{\pRT}{\texttt{pRT}\xspace}$
$\newcommand{\hydra}{\texttt{HyDRA}\xspace}$
$\newcommand{\platon}{\texttt{PLATON}\xspace}$
$\newcommand{\microns}{\textmu m\xspace}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Tracing the formation and migration history: molecular signatures in the atmosphere of misaligned hot Jupiter WASP-94Ab using JWST NIRSpec/G395H

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.11224-b31b1b.svg)](https://arxiv.org/abs/2505.11224)<mark>Appeared on: 2025-05-19</mark> -  _19 pages, 15 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

<mark>E.-M. Ahrer</mark>, et al.

</div>
<div id="abstract">

**Abstract:** The discovery of hot Jupiters that orbit very close to their host stars has long challenged traditional models of planetary formation and migration. Characterising their atmospheric composition --- mainly in the form of the carbon-to-oxygen (C/O) ratio and metallicity --- can provide insights into their formation locations and evolution pathways. With JWST we can characterise the atmospheres of these types of planets more precisely than previously possible, primarily because it allows us to determine both their atmospheric oxygen and carbon composition.Here, we present a JWST NIRSpec/G395H transmission spectrum from 2.8 -- 5.1 $\microns$ of WASP-94Ab, an inflated hot Jupiter with a retrograde misaligned orbit around its F-type host star. We find a relatively cloud-free atmosphere, with absorption features of $\ce{H2O}$ and $\ce{CO2}$ at detection significances of $\sim 4\sigma$ and $\sim 11\sigma$ , respectively. In addition, we detect tentative evidence of $\ce{CO}$ absorption at $\sim3\sigma$ , as well as hints of sulphur with the detection of $\ce{H2S}$ at a $\sim 2.5\sigma$ confidence level. Our favoured equilibrium chemistry model determines a C/O ratio of $0.49^{+0.08}_{-0.13}$ for WASP-94Ab's atmosphere, which is substellar compared to the star's C/O ratio of $0.68 \pm 0.10$ . The retrieved atmospheric metallicity is similar to the star's metallicity as both are $\sim 2\times$ solar.We find that this sub-stellar C/O ratio and stellar metallicity can be best explained by pebble accretion or planetesimal accretion in combination with large-distance migration of the planet.

</div>

<div id="div_fig1">

<img src="tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_Tiberius_Eureka_exotic_R100_with_residuals.png" alt="Fig11.1" width="50%"/><img src="tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_Tiberius_Eureka_exotic_R400_with_residuals.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** _Top panels: _Transmission spectrum of $\myplanet$ using JWST's NIRSpec/G395H, using three independent reduction pipelines, $\tiberius$(green), $\eureka$(dark blue) and $\exoticjedi$(orange). The two panels show the spectra at two different binning schemes with $R=100$ on the left and $R=400$ on the right. _Bottom panels:_ Differences between the reductions compared to \texttt{Eureka!}. Note that the y axis on the left panel is a different scale (-150,+150) ppm compared to the right panel (-400,+400) ppm for visual clarity. $\exoticjedi$ displays a slight offset compared to $\tiberius$ and $\eureka$ in the NRS1 spectrum.   (*fig:transmission-spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_models_with_contributions.png" alt="Fig7.1" width="50%"/><img src="tmp_2505.11224/./figures/WASP-94Ab_transmission_spectrum_models_with_contributions_eq_chem.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** $\eureka$ transmission spectrum of $\myplanet$ with the best-fit $\pRT$ models using free chemistry and equilibrium chemistry and the fitted corresponding spectral contributions on the top and bottom panels, respectively. The best-fit offset from the latter retrieval is applied to the NRS2 spectrum (87ppm, consistent within $0.3\sigma$ with the 92ppm from the free chemistry run). By eye the two models (equilibrium chemistry vs free chemistry) show remarkable agreement with the exception of the cloud-deck which is at much higher altitudes for the equilibrium chemistry (see bottom panel), while the free chemistry retrieval prefers a cloud deck at lower altitudes not shown in the top panel as it is off the y axis (equivalent of $\sim 1.05 \%$ transit depth). However, the equilibrium chemistry retrieval runs were preferred across the board when comparing their Bayesian evidence values for all reductions and resolutions, see Table \ref{tab:bayesian_evidences}. Therefore, our favoured model is the equilibrium model, with $\Delta \ln \mathcal{Z} = 3.7$($3.2\sigma$) in the case of $\eureka$ R=400 using $\pRT$.   (*fig:spectrum-free_chem-vs-eq_chem*)

</div>
<div id="div_fig3">

<img src="tmp_2505.11224/./figures/WASP-94Ab_comparison_JWST_HJs.png" alt="Fig9" width="100%"/>

**Figure 9. -** $\myplanet$ in context with JWST transmission spectra published for hot Jupiters ($\revision${here assumed as exoplanets with} temperatures $1,000$ K -- $2,000$ K, radii R$_\mathrm{p} > 0.5$ R$_\mathrm{Jup}$, masses M$_\mathrm{p} > 0.2$ M$_\mathrm{Jup}$) in the 2.5 -- 5.2 $\microns$ wavelength range: WASP-39b \citepalias{Carter2024AWASP-39b}, HD 189733b  (Fu2024HydrogenExoplanet) , HD 209458b  (Xue20242) , and WASP-15b  ([ and Kirk 2025](https://ui.adsabs.harvard.edu/abs/2025MNRAS.537.3027K)) . The colours represent the order of temperatures, from light to dark: WASP-39b ($\sim1170$ K), HD 189733b ($\sim1200$ K), HD 209458b ($\sim1460$ K),  WASP-94Ab ($\sim1500$ K; best-fitting offset applied), and WASP-15b ($\sim1680$ K). The scale heights are computed using a mean molecular mass of $\mu = 2.3$ amu and normalised using the minimum of the transit depth.   (*fig:jwst-HJs-transmission-spectra*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.11224"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xiion}{\xi_{\rm{ion}}}$
$\newcommand{\halpha}{H\ensuremath{\alpha}}$
$\newcommand{\hbeta}{H\ensuremath{\beta}}$
$\newcommand{\mstar}{\ensuremath{\log(M_{\rm{\star}}/M_{\rm{\odot}})}}$
$\newcommand{\orcidauthor}[3]{\author{\href{http://orcid.org/#1}{#2^{#3}}}}$
$\newcommand{\nion}[2]{#1 \textsc{#2}}$</div>



<div id="title">

# $\vspace{-1cm}$ A Cosmic Miracle: A Remarkably Luminous Galaxy at $\MakeLowercase{z}_{\MakeLowercase{\rm{spec}}}=14.44$ Confirmed with JWST $\vspace{-1.8cm}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.11263-b31b1b.svg)](https://arxiv.org/abs/2505.11263)<mark>Appeared on: 2025-05-19</mark> -  _Submitted to the Open Journal of Astrophysics. Comments greatly appreciated and warmly welcomed!_

</div>
<div id="authors">

R. P. Naidu, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** JWST has revealed a stunning population of bright galaxies at surprisingly early epochs, $z>10$ , where few such sources were expected. Here we present the most distant example of this class yet -- MoM-z14, a luminous ( $M_{\rm{UV}}=-20.2$ ) source in the COSMOS legacy field at $z_{\rm{spec}}=14.44^{+0.02}_{-0.02}$ that expands the observational frontier to a mere 280 million years after the Big Bang. The redshift is confirmed with NIRSpec/prism spectroscopy through a sharp Lyman- $\alpha$ break and $\approx3\sigma$ detections of five rest-UV emission lines. The number density of bright $z_{\rm{spec}}\approx14-15$ sources implied by our "Mirage or Miracle" survey spanning $\approx350$ arcmin $^{2}$ is $>100\times$ larger ( $182^{+329}_{-105}\times$ ) than pre-JWST consensus models. The high EWs of UV lines ( ${\approx}15{-}35$ Å) signal a rising star-formation history, with a ${\approx}10\times$ increase in the last 5 Myr ( $\rm{SFR_{\rm{5Myr}}}/\rm{SFR_{\rm{50Myr}}}=9.9^{+3.0}_{-5.8}$ ). The source is extremely compact (circularized $r_{\rm{e}} = 74^{+15}_{-12}$ pc), and yet resolved, suggesting an AGN is not the dominant source of light. The steep UV slope ( $\beta=-2.5^{+0.2}_{-0.2}$ ) implies negligible dust attenuation and a young stellar population. The absence of a strong damping wing may indicate that the immediate surroundings of MoM-z14 are partially ionized at a redshift where virtually every reionization model predicts a $\approx100\%$ neutral fraction. The nitrogen emission and highly super-solar [ N/C ] $>1$ hint at an abundance pattern similar to local globular clusters that may have once hosted luminous supermassive stars. Since this abundance pattern is also common among the most ancient stars born in the Milky Way, we may be directly witnessing the formation of such stars in dense clusters, connecting galaxy evolution across the entire sweep of cosmic time.

</div>

<div id="div_fig1">

<img src="tmp_2505.11263/./figs/z_summary.png" alt="Fig6" width="100%"/>

**Figure 6. -** ** Summary of spectroscopic redshift constraints.****Top-left:** We use the sharp Lyman-$\alpha$ break to derive the redshift ($z_{\rm{break}}$). Our best-fit model (hot pink) accounts for the IGM neutral fraction along the line of sight ($\rm{x}_{\rm{HI}}$) as well as the shape and normalization of the spectrum ($\beta_{\rm{UV}}$, $M_{\rm{UV}}$). See \S\ref{sec:lyaz} for details. **Top-right:** We are also able to determine the redshift by fitting the rest-UV emission lines that are detected in this source. Each individual line or blend is detected at $\approx3\sigma$. Collectively, this array of lines is detected at $\approx6\sigma$ resulting in an extremely precise redshift. **Bottom-left:** The photometric redshift distribution derived from NIRCam (silver) shows multiple peaks at $z<5$("Schrodinger galaxy"-like solutions; \citealt{Naidu22Schro}) and a dominant $z>10$ solution that led us to target this source. **Bottom-right:** Comparison of the break redshift and UV line redshift posteriors. Only a handful of galaxies of comparable luminosity at $z>10$ have shown a UV spectrum with multiple lines allowing for a precise redshift determination (\citealt{Bunker23,Castellano24}; both interestingly point-like sources interpreted as putative AGN). Typically, only break redshifts with relatively wide posteriors as shown in pink have been possible \citep[e.g.,][]{Curtis-Lake23, Wang23UNCOVERz12, Carniani24}.  (*fig:specz*)

</div>
<div id="div_fig2">

<img src="tmp_2505.11263/./figs/Re_v_Redshift_combo_v2.png" alt="Fig8" width="100%"/>

**Figure 8. -** ** A size-abundance bimodality among bright $z>10$ galaxies.**Figure adapted from \citet{Harikane2025UVLF} who reported this trend (see also \citealt[][]{Schaerer24}). Two types of sources are evident in the size-redshift plane (left) and size-EW plane (right) -- compact $\nion${N}{iv}] emitters such as MoM-z14 (purple, navy) and extended sources with weak $\nion${N}{iv}] such as JADES-GS-z14-0 (orange). The extrapolated size evolution from \citet{Shibuya15}(silver band) is shown in the left panel for guidance. MoM-z14 joins GNz11 and GLASSz12/GHz2 as extremely compact outliers at $z>10$, with sizes $<5-10\times$ what would be expected from the scaling relation. The measurements shown here are compiled from the following sources: GNz11 \citep[][]{Tacchella23GNz11, Bunker23}, GLASS-z12/GHz2 \citep[][]{Castellano24}, GNz9p4 \citep[][]{Schaerer24}, JADES-GS-z14-0 \citep[][]{Carniani24}, Maisie's Galaxy \citep[][]{Finkelstein23, ArrabalHaro23}, RXCJ2248-ID \citep[][]{Topping24}, Gz9p3 \citep[][]{Boyett24}, and CEERS2-588 \citep[][]{Harikane24specuvlf}.  (*fig:bimodal*)

</div>
<div id="div_fig3">

<img src="tmp_2505.11263/./figs/cue_posterior_v2.png" alt="Fig10" width="100%"/>

**Figure 10. -** **Emission line modeling results using \texttt{Cue**}\citep[][]{Li24cue}. MoM-z14's UV spectrum, with hints of several emission lines, provides a unique window into the physics of bright galaxies at cosmic dawn -- the ionizing sources powering them ($\log \xi_{\rm{ion}}$), the state of the gas in their ISM ($\log n_{\rm{H}}$, $\log U$), and their chemical abundance patterns ([O/H], [N/O], [C/O]). Satisfactory fits to the emission lines are shown in the top-right panel, while the corner-plot illustrates the inferred posteriors. Even with only SNR$\approx3$ emission lines, and despite the low-resolution of the data (e.g., $\nion${He}{ii}$\lambda1640$Å+$\nion${O}{iii}]$\lambda1661,1666$Å are observed as a blend), the model is not entirely unconstrained. Consistent with the lack of neutral gas around the source, a highly ionizing radiation field is inferred (e.g., $\log \xi_{\rm{ion}}/\rm{erg s^{-1}}\approx26.3$). There are also indications of a super-solar [N/O] and sub-solar [C/O] abundance pattern already in place by $z=14.44$ reminiscent of GN-z11 \citep[e.g.,][]{Cameron23} and other recently discovered N-emitters \citep[e.g.,][]{Schaerer24}.
     (*fig:cue*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.11263"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

493  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
